[View in Colaboratory](https://colab.research.google.com/github/ibraheem-moosa/protein-bvalue-prediction/blob/torch_rnn_gpu/Protein_B_value_prediction_LSTM.ipynb)

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch

In [53]:
accelerator

'cu80'

In [54]:
platform

'cp36-cp36m'

In [55]:
torch.__version__

'0.4.0'

In [56]:
torch.cuda.device_count()

1

In [57]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [0]:
from torch import nn
from torch.nn.functional import relu
from torch.nn.functional import leaky_relu
from torch.nn.functional import dropout
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import torch.utils.data
from scipy.sparse import load_npz
from numpy import load
from bisect import bisect
import matplotlib.pyplot as plt
from math import sqrt
import torch.optim as optim
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
import sys
import time
import os
import random

In [0]:
def summarize_tensor(tensor):
    return torch.max(tensor).item(), torch.min(tensor).item(), torch.mean(tensor).item(), torch.std(tensor).item()

In [0]:
  def close_event():
    plt.close()

In [0]:
def plot_true_and_prediction(y_true, y_pred):
    fig = plt.figure()
    timer = fig.canvas.new_timer(interval=10000)
    timer.add_callback(close_event)
    plt.title('Bidirectional, 8 Hidden States, 2 Output Layers')
    plt.plot(y_pred, 'y-')
    plt.plot(y_true, 'g-')
    timer.start()
    plt.show()

In [0]:
def summarize_nn(net):
    print('##############################################################')
    for name, param in net.named_parameters():
        print('---------------------------------------------------------------')
        print(name)
        print(summarize_tensor(param))
    print('##############################################################')

In [0]:
def get_avg_pcc(net, dataset, indices):
    pcc = []
    for i in indices:
        x, y = dataset[i]
        y_pred = net.predict(x)
        y, y_pred = y.cpu(), y_pred.cpu()
        for j in range(x.shape[0]):
            pcc.append(pearsonr(y_pred.numpy()[j].flatten(), y.numpy()[j].flatten())[0])

    pcc = np.array(pcc)
    pcc[np.isnan(pcc)] = 0
    return np.mean(pcc)

In [0]:
def cross_validation(net, dataset, indices, k, threshold):
    n = len(indices) // k
    r = len(indices) - n * k
    fold_lengths = [n + 1] * r + [n] * (k - r)
    cumulative_fl = [0]
    for fl in fold_lengths:
        cumulative_fl.append(cumulative_fl[-1] + fl)
    scores = []
    for i in range(k):
        print('Cross Validation Fold: {}'.format(i))
        train_indices = []
        validation_indices = []
        for j in range(k):
            if j == i:
                validation_indices.extend(indices[cumulative_fl[j]:cumulative_fl[j+1]])
            else:
                train_indices.extend(indices[cumulative_fl[j]:cumulative_fl[j+1]])
        train_pccs, validation_pccs = net.train(dataset, train_indices, validation_indices) 
        validation_pcc = max(validation_pccs)
        scores.append(validation_pcc)
        if validation_pcc < threshold:
            break
    return scores

In [0]:
def get_param_config(param_grid, keys):
    if len(keys) == 0:
        yield None
    else:
        for value in param_grid[keys[0]]:
            for rest_config in get_param_config(param_grid, keys[1:]):
                yield keys[0], value, rest_config

In [0]:
def gridsearchcv(net, dataset, indices, k, threshold, param_grid, param_set_funcs):
    result = []
    num_of_params = len(param_grid)
    for param_config in get_param_config(param_grid, list(param_grid.keys())):
        next_param_config = param_config
        param_config_dict = dict()
        while True:
            key, value, next_param_config = next_param_config
            param_config_dict[key] = value
            param_set_funcs[key](net, value)
            if next_param_config is None:
                break
            
        print('Running CV for params {}'.format(param_config_dict))
        scores = cross_validation(net, dataset, indices, k, threshold)
        mean_score = sum(scores) / len(scores)
        print('Got score {} for params {}'.format(mean_score, param_config_dict))
        result.append((param_config_dict, mean_score))
    return result

In [0]:
class ProteinDataset:

    def __init__(self, files, batch_size):
        self._Xes = []
        self._yes = []
        for xf,yf in files:
            X = torch.from_numpy(load_npz(xf).toarray()).reshape((-1, 21))
            y = torch.from_numpy(load(yf)['y']).reshape((-1, 1))
            assert(X.shape[0] == y.shape[0])
            self._Xes.append(X)
            self._yes.append(y)

        self._Xes.sort(key=lambda x:x.shape[0], reverse=True)
        self._yes.sort(key=lambda y:y.shape[0], reverse=True)
        self._lengths = [x.shape[0] for x in self._Xes]

        X_batches = []
        y_batches = []
        lengths = []
        for i in range(0, len(files), batch_size):
            stride = min(len(files) - i, batch_size)
            lengths.append(self._lengths[i:i+stride])
            X_batches.append(
                        pad_sequence(self._Xes[i:i+stride], batch_first=True))
            y_batches.append(
                        pad_sequence(self._yes[i:i+stride], batch_first=True))

        self._Xes = X_batches
        self._yes = y_batches
        self._lengths = lengths

    def __getitem__(self, idx):
        return self._Xes[idx], self._yes[idx], self._lengths[idx]

    def __len__(self):
        return len(self._Xes)

In [0]:
class FixedWidthFeedForwardNeuralNetwork(nn.Module):
    def __init__(self, width, num_outputs, num_layers, activation):
        super(FixedWidthFeedForwardNeuralNetwork, self).__init__()
        self.linear_layers = [nn.Linear(width, width) for i in range(num_layers-1)]
        self.linear_layers.append(nn.Linear(width, num_outputs))
        self.activation = activation
        for i in range(num_layers):
            self.register_parameter('FF' + str(i) + '_weight_', self.linear_layers[i].weight)
            self.register_parameter('FF' + str(i) + '_bias_', self.linear_layers[i].bias)

    def forward(self, x):
        out = self.activation(self.linear_layers[0](x))
        for i in range(1, len(self.linear_layers) - 1):
            out = self.activation(self.linear_layers[i](out))
        out = self.linear_layers[-1](out)
        return out

In [0]:
class RecurrentNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size=8, output_layer_depth=1,  num_hidden_layers=1, hidden_scale=1.0, ff_scale=0.001, init_lr=1e-3, gamma=0.99, weight_decay=0.1, grad_clip=1.0):
        super(RecurrentNeuralNetwork, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_layer_depth = output_layer_depth
        self.num_hidden_layers = num_hidden_layers
        self.hidden_scale = hidden_scale
        self.ff_scale = ff_scale
        self.init_lr = init_lr
        self.gamma = gamma
        self.weight_decay = weight_decay
        self.grad_clip = grad_clip
        self.init_layers()

    def forward(self, x, lengths):
        packed_x = pack_padded_sequence(x, lengths, batch_first=True)
        packed_out, h = self.rnn_layer(packed_x)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)
        out = self.output_layer(out)
        return out

    def init_layers(self):
        self.rnn_layer = nn.RNN(input_size=self.input_size, 
                                hidden_size=self.hidden_size,
                                nonlinearity='relu',
                                num_layers=self.num_hidden_layers,
                                batch_first=True, 
                                bidirectional=True)
        self.output_layer = FixedWidthFeedForwardNeuralNetwork(self.hidden_size * 2, 1, self.output_layer_depth, leaky_relu)
        self._init_weights_()

    def _init_weights_(self):
        ff_init_method = nn.init.normal_
        hidden_weight_init_method = nn.init.eye_
        bias_init_method = nn.init.constant_
        for name, param in self.rnn_layer.named_parameters():
            if 'weight_hh' in name:
                hidden_weight_init_method(param)
                with torch.no_grad():
                    param.mul_(self.hidden_scale)
                param.requires_grad_()
            elif 'weight_ih' in name:
                ff_init_method(param, std=self.ff_scale)
            else:
                bias_init_method(param, 0)

        for name, param in self.output_layer.named_parameters():
            if 'weight' in name:
                ff_init_method(param, std=self.ff_scale)
            else:
                bias_init_method(param, 0)

    def predict(self, x, lengths):
        with torch.no_grad():
            out = self.forward(x, lengths)
            return out

    def reset_hidden_size(self, hidden_size):
        self.hidden_size = hidden_size
        self.init_layers()

    def reset_weight_decay(self, weight_decay):
        self.weight_decay = weight_decay
        self.init_layers()

    def reset_gamma(self, gamma):
        self.gamma = gamma
        self.init_layers()

    def reset_output_layer_depth(self, output_layer_depth):
        self.output_layer_depth = output_layer_depth
        self.init_layers()
    
    def reset_num_hidden_layers(self, num_hidden_layers):
        self.num_hidden_layers = num_hidden_layers
        self.init_layers()

    def train(self, dataset, validation_dataset, model_dir=None, patience=5, warm_start_last_epoch=-1):
        criterion = nn.MSELoss(size_average=False)
        optimizer = optim.Adam([{'params' : self.parameters(), 'initial_lr' : self.init_lr}], 
                        lr=self.init_lr, weight_decay=self.weight_decay, amsgrad=False)
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, self.gamma)
        self._init_weights_()
        num_of_batches = len(dataset)
        validation_num_of_batches = len(validation_dataset)
        
        best_epoch = 0
        best_mse = 1000.0
        validation_mses = []
        train_mses = []
        for epoch in range(warm_start_last_epoch + 1, warm_start_last_epoch + 1 + 1000):
            scheduler.step()
            running_loss = 0.0
            total_train_length = 0
            for i in range(num_of_batches):
                x, y, lengths = dataset[i]
                total_train_length += sum(lengths)
                optimizer.zero_grad()
                y_pred = self.forward(x, lengths)
                loss = criterion(y_pred, y)
                loss.backward()
                nn.utils.clip_grad_value_(self.parameters(), self.grad_clip)
                optimizer.step()
                running_loss += loss.item()
            running_loss /= total_train_length
            train_mses.append(running_loss)

            validation_loss = 0.0
            total_validation_length = 0
            for i in range(validation_num_of_batches):
                x, y, lengths = validation_dataset[i]
                total_validation_length += sum(lengths)
                y_pred = self.predict(x, lengths)
                loss = criterion(y_pred, y)
                validation_loss += loss.item()
            validation_loss /= total_validation_length
            validation_mses.append(validation_loss)

            if running_loss < best_mse:
                best_mse = running_loss
                best_epoch = epoch
                print(best_epoch)
            
            print('Epoch: {0:02d} Loss: {1:.6f} Validation Loss: {2:.6f} Time: {3}'.format(
                                    epoch, running_loss, validation_loss, time.strftime('%Y-%m-%d %H:%M:%S')))

            if model_dir is not None:
                torch.save(self.state_dict(), os.path.join(model_dir, 'net-{0:02d}'.format(epoch)))

            if epoch - best_epoch == patience:
                break

        return train_mses, validation_mses

In [0]:
class LSTMNeuralNetwork(RecurrentNeuralNetwork):
    def __init__(self, input_size, hidden_size=8, output_layer_depth=1,  num_hidden_layers=1, hidden_scale=1.0, ff_scale=0.001, init_lr=1e-3, gamma=0.99, weight_decay=0.1, grad_clip=1.0):
        super(LSTMNeuralNetwork, self).__init__(input_size, hidden_size, output_layer_depth, num_hidden_layers, hidden_scale, ff_scale, init_lr, gamma, weight_decay, grad_clip)

    def init_layers(self):
        self.lstm_layer = nn.LSTM(input_size=self.input_size, 
                                hidden_size=self.hidden_size,
                                #nonlinearity='relu',
                                num_layers=self.num_hidden_layers,
                                batch_first=True, 
                                bidirectional=True)
        self.output_layer = FixedWidthFeedForwardNeuralNetwork(self.hidden_size * 2, 1, self.output_layer_depth, leaky_relu)
        self._init_weights_()

    def _init_weights_(self):
        ff_init_method = nn.init.normal_
        hidden_weight_init_method = nn.init.eye_
        bias_init_method = nn.init.constant_
        for name, param in self.lstm_layer.named_parameters():
            if 'weight_hh' in name:
                hidden_weight_init_method(param)
                with torch.no_grad():
                    param.mul_(self.hidden_scale)
                param.requires_grad_()
            elif 'weight_ih' in name:
                ff_init_method(param, std=self.ff_scale)
            else:
                bias_init_method(param, 0)

        for name, param in self.output_layer.named_parameters():
            if 'weight' in name:
                ff_init_method(param, std=self.ff_scale)
            else:
                bias_init_method(param, 0)

    def forward(self, x, lengths):
        packed_x = pack_padded_sequence(x, lengths, batch_first=True)
        packed_out, h = self.lstm_layer(packed_x)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)
        out = self.output_layer(out)
        return out

In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
protein_list_file = drive.CreateFile({'id' : '1B8jJtU2sZGEZgI66Wn3qzjROG6Bqiusi'})

In [0]:
protein_list_file.GetContentFile('protein_list.txt')

In [0]:
rnn_inputs_compressed_file = drive.CreateFile({'id' : '1URxykv0RfgC3f0XJLteZrQ8lwTILgaZZ'})

In [0]:
rnn_inputs_compressed_file.GetContentFile('rnn_inputs.tar.gz')

In [0]:
!tar xzf rnn_inputs.tar.gz

In [79]:
torch.set_printoptions(precision=2, linewidth=140)
torch.manual_seed(42)

In [0]:
with open('protein_list.txt') as protein_list_file:
    protein_list = protein_list_file.read().split()
    protein_list = [s.upper().strip() for s in protein_list]

In [0]:
X_files = []
y_files = []

In [0]:
for protein in protein_list:
    X_files.append(os.path.join('rnn_inputs', 'X_' + protein + '_rnn_.npz'))
    y_files.append(os.path.join('rnn_inputs', 'y_' + protein + '_rnn_.npz'))

In [0]:
batch_size = 64

In [0]:
indices = list(range(len(protein_list)))
random.seed(42)
random.shuffle(indices)
train_indices = indices[:int(0.8 * len(indices))]
test_indices = indices[int(0.8 * len(indices)):]

In [85]:
files = list(zip(X_files, y_files))
dataset = ProteinDataset([files[i] for i in train_indices], batch_size)
print('Dataset init done ', len(dataset))
test_dataset = ProteinDataset([files[i] for i in test_indices], 64)
print('Test Dataset init done ', len(test_dataset))

Dataset init done  39
Test Dataset init done  10


In [0]:
indices = list(range(len(dataset)))
random.seed(42)
random.shuffle(indices)
#indices = indices[:100]
train_indices = indices[:int(0.8 * len(indices))]
validation_indices = indices[int(0.8 * len(indices)):]

In [0]:
warm_start_model_params = None
warm_start_last_epoch = -1

In [0]:
init_lr = 2.0 ** -10
momentum = 0.9
weight_decay = 1e-4
gamma = 0.999
hidden_size = 64
hidden_scale = 1.0
num_hidden_layers = 1
output_layer_depth = 8
ff_scale = 0.6
grad_clip = 10.0
nesterov = True

In [0]:
net = LSTMNeuralNetwork(21,
        hidden_size=hidden_size,
        num_hidden_layers=num_hidden_layers,
        output_layer_depth=output_layer_depth,
        hidden_scale=hidden_scale, ff_scale=ff_scale, 
        init_lr=init_lr, gamma=gamma, weight_decay=weight_decay)

In [0]:
!mkdir -p models

In [0]:
!ls models

In [0]:
net.train(dataset, test_dataset, patience=20, model_dir='models')

Epoch: 00 Loss: 744324120.975313 Validation Loss: 10545276.076249 Time: 2018-09-08 06:29:00
Epoch: 01 Loss: 5507755.982546 Validation Loss: 1400525.036001 Time: 2018-09-08 06:30:06
